In [2]:
%cd ..
%ls



/home/aoezkan/eth/dslab/dsl-hs-2024/src
networks.py      pass_tables.py         resp.py                utils.py
notebooks/       __pycache__/           rl/
pass_options.py  responsibility.ipynb*  skillcorner_parser.py


In [3]:

import gym
# plot xt_table
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np



In [4]:

env = gym.make("LunarLander-v2")  # A popular Box2D environment
print(env)


<TimeLimit<OrderEnforcing<PassiveEnvChecker<LunarLander<LunarLander-v2>>>>>


In [4]:

# Reset the environment to generate the first observation
observation, info = env.reset(seed=42)
for _ in range(100000):
    # this is where you would insert your policy
    action = env.action_space.sample()

    # step (transition) through the environment with the action
    # receiving the next observation, reward and if the episode has terminated or truncated
    observation, reward, terminated, truncated, info = env.step(action)

    # If the episode has ended then we can reset to start a new episode
    if terminated or truncated:
        observation, info = env.reset()

env.close()

In [12]:
from pass_options import attacker_options
import pandas as pd


WYSCOUT_ID = 5414111
SKILLCORNER_ID = 952209


DATA_PATH = f'../data/networks/match_{SKILLCORNER_ID}/'
PASSES_DF_PATH = DATA_PATH + 'passes_df.csv'

DATA_PATH_PASSES = f'../data/'
XT_PLOT_PATH = DATA_PATH_PASSES + 'smoothed_xt.csv'


passes_df = pd.read_csv(PASSES_DF_PATH)


# load xt table
xt_table = pd.read_csv(XT_PLOT_PATH)

cell_width = 100 / xt_table.shape[1]
cell_height = 100 / xt_table.shape[0]

# prepare with pitch_dict, outside the function
pitch_length = 105
pitch_width = 68
xt_rows, xt_cols = 68, 105
cell_width = pitch_length / xt_cols
cell_height = pitch_width / xt_rows

pitch_dict = {
    'pitch_length': pitch_length,
    'pitch_width': pitch_width,
    'xt_rows': xt_rows,
    'xt_cols': xt_cols,
    'cell_width': cell_width,
    'cell_height': cell_height,
    'xt_table': xt_table
}

print(pitch_dict['pitch_length'], pitch_dict['pitch_width'], pitch_dict['xt_rows'], pitch_dict['xt_cols'], pitch_dict['cell_width'], pitch_dict['cell_height'])

105 68 68 105 1.0 1.0


In [17]:
print(pitch_dict['pitch_length']/2, pitch_dict['pitch_width']/2)

print(passes_df['location.x'].max(), passes_df['location.x'].min())
print(passes_df['location.y'].max(), passes_df['location.y'].min())

52.5 34.0
49.75 -49.75
33.27157894736842 -31.814736842105265


In [23]:
frame_ids = passes_df['frame'].unique()

print(len(frame_ids))

# sample randomly frame from the frame_ids
np.random.choice(frame_ids)

893


16935

In [1]:
import numpy as np
import pandas as pd

WYSCOUT_ID = 5414111
SKILLCORNER_ID = 952209

NETWORKS_PATH = f'../../data/networks/match_{SKILLCORNER_ID}/'
PASSES_DF_PATH = NETWORKS_PATH + 'passes_df.csv'

DATA_PATH= f'../../data/'
XT_PLOT_PATH = DATA_PATH + 'smoothed_xt.csv'
METADATA_PATH = DATA_PATH + f'skillcorner/{SKILLCORNER_ID}_metadata.csv'

passes_df = pd.read_csv(PASSES_DF_PATH)
xt_table = pd.read_csv(XT_PLOT_PATH)

cell_width = 100 / xt_table.shape[1]
cell_height = 100 / xt_table.shape[0]

# prepare with pitch_dict, outside the function
pitch_length = 105
pitch_width = 68
xt_rows, xt_cols = 68, 105
cell_width = pitch_length / xt_cols
cell_height = pitch_width / xt_rows


pitch_dict = {
    'pitch_length': pitch_length,
    'pitch_width': pitch_width,
    'xt_rows': xt_rows,
    'xt_cols': xt_cols,
    'cell_width': cell_width,
    'cell_height': cell_height,
    'xt_table': xt_table,
    'xt_table_np': xt_table.to_numpy(),
}




import gym

from gym.envs.registration import register



register(
    id='DefenderPosEnv',  # Unique identifier
    entry_point='env:DefenderPosEnv',  # Specify the path to your environment class
    # entry_point='rl.env:DefenderPosEnv',  # Specify the path to your environment class
)

env = gym.make("DefenderPosEnv", pitch_dict=pitch_dict, passes_df=passes_df)



/Users/alpayozkan/opt/anaconda3/envs/socc/lib/python3.8/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/Users/alpayozkan/opt/anaconda3/envs/socc/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (22, 2)
  logger.warn(


In [2]:
from stable_baselines3 import PPO  # Example algorithm: Proximal Policy Optimization
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback


# Initialize the RL model
model = PPO("MlpPolicy", env, verbose=1)  # "MlpPolicy" uses a fully connected network

eval_callback = EvalCallback(
    env, 
    best_model_save_path='./logs/', 
    log_path='./logs/', 
    eval_freq=1000, 
    deterministic=True, 
    render=True
)


import sys
# sys.stdout = open('out.txt', 'w')

# Train the model
timesteps = 10000
# model.learn(total_timesteps=timesteps, callback=eval_callback)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/Users/alpayozkan/opt/anaconda3/envs/socc/lib/python3.8/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/Users/alpayozkan/opt/anaconda3/envs/socc/lib/python3.8/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [ ]:
# Evaluate the trained model
# mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)

# print(f"Mean reward: {mean_reward}, Std reward: {std_reward}")

# Test the trained model
obs, _ = env.reset()
for step in range(10000):
    action, _states = model.predict(obs, deterministic=True)  # Choose the best action
    obs, reward, done, truncated, info = env.step(action)
    print('reward: ', reward)
    # env.render()  # Render the environment
    if done or truncated:
        obs, _ = env.reset()

env.close()

/Users/alpayozkan/opt/anaconda3/envs/socc/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:141: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/Users/alpayozkan/opt/anaconda3/envs/socc/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
/Users/alpayozkan/opt/anaconda3/envs/socc/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/alpayozkan/opt/anaconda3/envs/socc/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:141: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be float32, actual type: flo

frame_id: 11367
[[-41.16         1.11      ]
 [-18.46259359  -5.72      ]
 [-18.67614917   5.36000642]
 [-16.44847948 -14.68      ]
 [-15.847442    18.08001251]
 [ -9.78696823   5.31002366]
 [-10.51        13.17      ]
 [ -3.16838184  -9.55      ]
 [ -7.7        -17.2       ]
 [ -4.80281266  21.03000736]
 [ -1.07         4.84      ]
 [ 33.42         1.47      ]
 [  7.45         1.32      ]
 [  5.36       -14.73      ]
 [ -9.44        33.17      ]
 [ -5.68       -25.14      ]
 [-12.44       -13.69      ]
 [ -8.79         5.43      ]
 [ -8.7         14.25      ]
 [-13.79        -9.27      ]
 [-17.39         2.64      ]
 [  4.8         14.67      ]]
[[0.0044526  0.        ]
 [0.         0.00740641]
 [0.00166608 0.00385083]
 [0.         0.00152052]
 [0.00489239 0.00255803]
 [0.00780467 0.00303187]
 [0.         0.        ]
 [0.         0.00161816]
 [0.         0.        ]
 [0.00102343 0.00718735]
 [0.         0.        ]]
reward:  -0.0024179072371686983
[[-41.16         1.11      ]
 [-18.45

In [5]:
state

NameError: name 'state' is not defined

In [2]:

state = env.reset()
done = False

prev_reward = -1
count=0
while not done:
    action = env.action_space.sample()  # Random action
    next_state, reward, done, term, info = env.step(action)
    print(f"Reward: {reward}")

    # if reward == prev_reward:
    #     count += 1
    #     if count == 10:
    #         break
    # else:
    #     count = 0
    # prev_reward = reward
    # print(f"State: {next_state}, Reward: {reward}, Done: {done}")

/home/aoezkan/miniconda3/envs/socc/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:141: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/home/aoezkan/miniconda3/envs/socc/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [7]:
state

(array([[ 53.19600172, -29.44854315],
        [  4.75975265, -13.32733114],
        [-20.29444066, -17.66100207],
        [  6.28373004, -22.74819743],
        [ 22.15528885, -31.46049249],
        [  4.44395323, -21.04835705],
        [-48.96145514,   1.06483843],
        [ 11.35049379,  31.09060318],
        [  7.77271598,  10.0890535 ],
        [-46.07695109, -45.70541209],
        [ 16.38750649,   3.44122141],
        [-42.36      ,   3.59      ],
        [-27.85      ,   2.91      ],
        [-19.57      , -14.82      ],
        [-13.44      ,  31.94      ],
        [ -5.44      ,  16.53      ],
        [ -8.39      ,  -2.76      ],
        [  1.34      ,   2.99      ],
        [  5.78      , -16.56      ],
        [  6.        ,  31.52      ],
        [  3.67      ,  10.56      ],
        [-24.19      ,  19.91      ]]),
 {})

In [8]:
prev_reward

-0.0044683237693139005

In [10]:
np.allclose(prev_reward, prev_reward, atol=1e-4)

True

In [4]:
len(env.step(action))

[  0.3  -32.05]
(2,)


5

In [7]:
reward == prev_reward

False

In [8]:
prev_reward

-1

/home/aoezkan/miniconda3/envs/socc/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:141: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/home/aoezkan/miniconda3/envs/socc/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [8]:
%ls

networks.py      __pycache__/           skillcorner_parser.py
notebooks/       responsibility.ipynb*  utils_parallel.py
pass_options.py  resp.py                utils.py
pass_tables.py   rl/
